# Integrar com o Drive


# Resgatar repositorio Git

---






In [9]:
!git clone https://github.com/ming053l/DRCT.git
%cd DRCT

Cloning into 'DRCT'...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 458 (delta 28), reused 2 (delta 0), pack-reused 406 (from 1)
Receiving objects: 100% (458/458), 8.50 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (255/255), done.
/content/DRCT


# Instalar dependencias


In [16]:
!pip install basicsr==1.3.4.9
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt
!python setup.py develop
!pip install gdown

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1.2+rocm5.6, 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.

# Criar diretorios de entrada e saida


In [11]:
import os

os.makedirs("inputs_imgs", exist_ok=True)
os.makedirs("outputs", exist_ok=True)
os.makedirs("experiments/pretrained_models", exist_ok=True)

# 📥 Etapa 3: Baixar modelo Real-DRCT-GAN_Finetuned (via gdown)

Baixar modelo pré treinado de sua escolha: [acesse aqui](https://drive.google.com/drive/folders/1QJHdSfo-0eFNb96i8qzMJAPw31u9qZ7U)

In [12]:
# 📥 Etapa 3: Baixar modelo Real-DRCT-GAN_Finetuned (via gdown)
import gdown

model_url = "https://drive.google.com/uc?id=1ujETEVfuQXealAAg6YV8fKsHplGdqF0G"
gdown.download(model_url, "experiments/pretrained_models/Real-DRCT-GAN_Finetuned.pth", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ujETEVfuQXealAAg6YV8fKsHplGdqF0G
From (redirected): https://drive.google.com/uc?id=1ujETEVfuQXealAAg6YV8fKsHplGdqF0G&confirm=t&uuid=17d1f11c-7faf-4a30-b24a-6aa577053013
To: /content/DRCT/experiments/pretrained_models/Real-DRCT-GAN_Finetuned.pth
100%|██████████| 246M/246M [00:01<00:00, 138MB/s]


'experiments/pretrained_models/Real-DRCT-GAN_Finetuned.pth'


# 📥 Etapa 4: Baixar imagem de teste

In [13]:
img_url = "https://drive.google.com/uc?id=1B4Qafq44qYE1G838DW4TnjirIztFBEbb"
gdown.download(img_url, "inputs_imgs/test.png", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B4Qafq44qYE1G838DW4TnjirIztFBEbb
To: /content/DRCT/inputs_imgs/test.png
100%|██████████| 71.3k/71.3k [00:00<00:00, 2.61MB/s]


'inputs_imgs/test.png'

# 📄 Cria o YAML de configuração

In [26]:
yml = """
name: DRCT_Custom_Test
model_type: DRCTModel
scale: 4
num_gpu: 0
manual_seed: 0

datasets:
  test_1:
    name: MyTestImages
    type: SingleImageDataset
    dataroot_gt: ~
    dataroot_lq: ./inputs_imgs
    io_backend:
      type: disk

network_g:
  type: DRCT
  upscale: 4
  in_chans: 3
  img_size: 64
  window_size: 16
  compress_ratio: 3
  squeeze_factor: 30
  conv_scale: 0.01
  overlap_ratio: 0.5
  img_range: 1.
  depths: [6, 6, 6, 6, 6, 6]
  embed_dim: 180
  num_heads: [6, 6, 6, 6, 6, 6]
  mlp_ratio: 2
  upsampler: 'pixelshuffle'
  resi_connection: '1conv'

path:
  pretrain_network_g: ./experiments/pretrained_models/Real-DRCT-GAN_Finetuned.pth
  strict_load_g: true
  param_key_g: 'params_ema'

val:
  save_img: true
  suffix: ~
  metrics: {}
"""

os.makedirs("options/test", exist_ok=True)
with open("options/test/DRCT_custom_test.yml", "w") as f:
    f.write(yml)

# Rodar inferência


In [27]:
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py
!python drct/test.py -opt options/test/DRCT_custom_test.yml

Disable distributed.
Path already exists. Rename it to /content/DRCT/results/DRCT_Custom_Test_archived_20250419_010032
2025-04-19 01:00:32,794 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.3.4.9
	PyTorch: 2.6.0+cu124
	TorchVision: 0.21.0+cu124
2025-04-19 01:00:32,795 INFO: 
  name: DRCT_Custom_Test
  model_type: DRCTModel
  scale: 4
  num_gpu: 0
  manual_seed: 0
  datasets:[
    test_1:[
      nam